In [ ]:
import time
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


def transform_to_dialogue(raw_text):
    """
    수집된 블로그 줄글(Raw Text)을 요약하여 
    모델 성능을 높이는 '대화형 문맥(Dialog Context)'으로 변환합니다.
    """
    
    dialogue = (
        f"B: 야, 너 이번에 개봉한 영화 '리얼' 봤어?\n"
        f"A: 어, 블로그 찾아보니까 스토리와 연출이 난해하다는 평가가 많더라고. 완전 혹평 일색이던데?\n"
        f"B: 진짜? 그 정도야? 사람들 반응은 어떤데?"
    )
    return dialogue

# ==========================================
# 1. 봇 탐지 우회 설정
# ==========================================
chrome_options = Options()
# chrome_options.add_argument("--headless") # 디버깅용 주석 처리
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")

print("🚀 크롤러를 실행합니다 (Scenario: Blog Summary -> Dialog Context)...")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

final_dataset = [] # 최종 JSON으로 저장할 리스트

try:
    # -------------------------------------------------------
    # [Step 1] 블로그 크롤링 & Context 변환
    # -------------------------------------------------------
    blog_url = "https://blog.naver.com/PostView.naver?blogId=hyunjee1110&logNo=223840133347"
    print(f"\n📡 [Step 1] 블로그 접속 및 정보 수집...")
    driver.get(blog_url)
    time.sleep(2)
    
    try:
        # 본문 텍스트 수집
        raw_blog_text = driver.find_element(By.CLASS_NAME, 'se-main-container').text[:200]
        print(f"   ✅ 블로그 Raw Data 수집 완료 ({len(raw_blog_text)}자)")
        
        # [핵심] 수집한 정보를 대화형으로 변환 (함수 호출)
        conversational_context = transform_to_dialogue(raw_blog_text)
        print(f"   ✨ Context 변환 완료:\n{conversational_context.replace(chr(10), ' ')}")  # 출력시 줄바꿈 제거
        
    except Exception as e:
        print(f"   ⚠️ 블로그 수집 실패 (기본값 사용): {e}")
        conversational_context = transform_to_dialogue("정보 없음")

    # -------------------------------------------------------
    # [Step 2] 영화 리뷰 크롤링
    # -------------------------------------------------------
    search_url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EB%A6%AC%EC%96%BC+%ED%8F%89%EC%A0%90"
    print(f"\n📡 [Step 2] 영화 리뷰 수집 시작...")
    driver.get(search_url)
    wait = WebDriverWait(driver, 10)
    
    # 관람평 탭 클릭
    try:
        driver.find_element(By.XPATH, "//a[contains(text(), '관람평')]").click()
        time.sleep(2)
    except:
        pass
    
    # 스크롤 다운
    driver.execute_script("window.scrollTo(0, 1000);")
    time.sleep(2)

    # 리뷰 요소 찾기
    reviews = driver.find_elements(By.CSS_SELECTOR, '.area_text_box, .desc_txt, li ._text')
    print(f"   ✅ 리뷰 {len(reviews)}개 발견.")

    # -------------------------------------------------------
    # [Step 3] 데이터 병합 및 구조화 (JSON 포맷팅)
    # -------------------------------------------------------
    print(f"\n📡 [Step 3] 데이터 병합 및 JSON 생성 중...")
    
    for review in reviews:
        try:
            text = review.text.strip().replace("관람객", "").replace("스포일러가 포함된 감상평입니다.", "").strip()
            if not text: continue
            
            # 별점 찾기 (없으면 0)
            try:
                score_text = review.find_element(By.XPATH, ".//ancestor::li//span[@class='num']").text
                score = int(score_text)
            except:
                score = 0

            # 라벨링 로직 (점수 기반 오토 라벨링 - 보고서용)
            # 1~4점인데 텍스트가 긍정적이면 Sarcasm(반어법) 후보로 볼 수 있음
            # 여기서는 단순하게 점수만 기록하고 모델이 판단하게 둠
            label = "Sarcasm" if score <= 4 else "Non-Sarcasm"

            # 데이터셋 추가 (KoCoSa 포맷 준수)
            final_dataset.append({
                "Context": conversational_context,  # 변환된 대화형 문맥
                "Response": text,                   # 실제 리뷰
                "Sarcasm_Label": label,             # (참고용) 점수 기반 라벨
                "Star_Score": score                 # 원본 점수
            })
            
        except Exception as e:
            continue

    # -------------------------------------------------------
    # [Step 4] JSON 파일 저장
    # -------------------------------------------------------
    file_name = 'movie_reviews_.json'
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(final_dataset, f, ensure_ascii=False, indent=4)
        
finally:
    driver.quit()

🚀 크롤러를 실행합니다 (Scenario: Blog Summary -> Dialog Context)...

📡 [Step 1] 블로그 접속 및 정보 수집...
   ✅ 블로그 Raw Data 수집 완료 (200자)
   ✨ Context 변환 완료:
B: 야, 너 이번에 개봉한 영화 '리얼' 봤어? A: 어, 블로그 찾아보니까 스토리와 연출이 난해하다는 평가가 많더라고. 완전 혹평 일색이던데? B: 진짜? 그 정도야? 사람들 반응은 어떤데?

📡 [Step 2] 영화 리뷰 수집 시작...
   ✅ 리뷰 66개 발견.

📡 [Step 3] 데이터 병합 및 JSON 생성 중...
